In [5]:
import sys, os
import numpy as np
import pandas as pd
from utils.utils import read_txt, read_txt_np_int, read_txt_split

# INPUT

In [20]:
inputfilename = './inputs/day6A.txt'

inputdata = read_txt(inputfilename)

In [21]:
map_matrix = np.array([list(x) for x in inputdata])
map_matrix

array([['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ...,
       ['.', '.', '.', ..., '.', '#', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.']], dtype='<U1')

In [23]:
map_matrix[0,1]

np.str_('.')

## PART 1

In [31]:
# Find guard postion
starting_guard_position = np.where(map_matrix == '^')
guard_position = (starting_guard_position[0][0], starting_guard_position[1][0])
print(map_matrix[guard_position])

# Map size
map_size = map_matrix.shape
print(map_size)


^
(130, 130)


In [35]:
guard_direction_shift = {'^': '>', 'v': '<', '<': '^', '>': 'v'}
movement_map = {'^': (-1, 0), 'v': (1, 0), '<': (0, -1), '>': (0, 1)}

def next_guard_position_direction(current_position, current_direction, map_size, map_matrix):
    movement = np.array(movement_map[current_direction])
    next_position = current_position + movement
    # Check if next position is within map bounds
    if (0 <= next_position[0] < map_size[0]) and (0 <= next_position[1] < map_size[1]):
        # Check map cell content
        if map_matrix[tuple(next_position)] == '#':
            # Change direction instead of moving
            new_direction = guard_direction_shift[current_direction]
            return current_position, new_direction  # Stay in place but change direction
        return next_position, current_direction  # Move to next position
    else:
        return None, None  # Out of bounds

In [36]:
visited_map = map_matrix.copy()
current_position = np.array(guard_position)
current_direction = '^'
while current_position is not None:
    # Mark the position as visited
    visited_map[tuple(current_position)] = 'X'
    current_position, current_direction = next_guard_position_direction(current_position, current_direction, map_size, map_matrix)


In [39]:
np.sum(visited_map == 'X')

np.int64(5199)

## PART 2

In [48]:
blocks_pos = np.where(map_matrix == '#')
len(blocks_pos[0]), len(blocks_pos[1])

(815, 815)

In [69]:
def get_first_block_in_direction(start_position, direction, map_size, blocks_pos):
    if direction == '^':
        # Get all in the same column and above the start position
        candidates = np.where((blocks_pos[1] == start_position[1]) & (blocks_pos[0] < start_position[0]))[0]
        if len(candidates) == 0:
            return None
        return candidates[-1]
    elif direction == 'v':
        # Get all in the same column and below the start position
        candidates = np.where((blocks_pos[1] == start_position[1]) & (blocks_pos[0] > start_position[0]))[0]
        if len(candidates) == 0:
            return None
        return candidates[0]
    elif direction == '<':
        # Get all in the same row and to the left of the start position
        candidates = np.where((blocks_pos[0] == start_position[0]) & (blocks_pos[1] < start_position[1]))[0]
        if len(candidates) == 0:
            return None
        return candidates[-1]
    elif direction == '>':
        # Get all in the same row and to the right of the start position
        candidates = np.where((blocks_pos[0] == start_position[0]) & (blocks_pos[1] > start_position[1]))[0]
        if len(candidates) == 0:
            return None
        return candidates[0]
    return None

In [93]:
def create_blocks_dict(blocks_pos):
    blocks_dict = {i: {'position': (blocks_pos[0][i], blocks_pos[1][i])}  for i in range(len(blocks_pos[0]))}
    for i, block in blocks_dict.items():
        block_position = block['position']
        visible_blocks = 0
        for direction in ['^', 'v', '<', '>']:
            previous_position = block_position - np.array(movement_map[direction])
            if (0 <= previous_position[0] < map_size[0]) and (0 <= previous_position[1] < map_size[1]):
                new_direction = guard_direction_shift[direction]
                next_block = get_first_block_in_direction(previous_position, new_direction, map_size, blocks_pos)
                block[direction] = next_block
    return blocks_dict

In [75]:
candidates = get_first_block_in_direction((40,40), '<', map_size, blocks_pos)
print(candidates)
print(blocks_pos[0][candidates], blocks_pos[1][candidates] )

265
40 14


In [94]:
blocks_dict = create_blocks_dict(blocks_pos)

In [95]:
map_size, blocks_dict

((130, 130),
 {0: {'position': (np.int64(0), np.int64(7)),
   '^': np.int64(9),
   '<': None,
   '>': np.int64(214)},
  1: {'position': (np.int64(0), np.int64(17)),
   '^': np.int64(9),
   '<': None,
   '>': np.int64(325)},
  2: {'position': (np.int64(0), np.int64(32)),
   '^': np.int64(10),
   '<': None,
   '>': np.int64(118)},
  3: {'position': (np.int64(0), np.int64(57)),
   '^': np.int64(10),
   '<': None,
   '>': np.int64(49)},
  4: {'position': (np.int64(0), np.int64(64)),
   '^': np.int64(10),
   '<': None,
   '>': np.int64(27)},
  5: {'position': (np.int64(0), np.int64(69)),
   '^': np.int64(11),
   '<': None,
   '>': np.int64(212)},
  6: {'position': (np.int64(0), np.int64(98)),
   '^': np.int64(11),
   '<': None,
   '>': np.int64(22)},
  7: {'position': (np.int64(0), np.int64(101)),
   '^': None,
   '<': None,
   '>': np.int64(52)},
  8: {'position': (np.int64(0), np.int64(117)),
   '^': None,
   '<': None,
   '>': np.int64(23)},
  9: {'position': (np.int64(1), np.int64(19)),

In [96]:
def get_full_route(initial_position, initial_direction, map_size, blocks_dict, blocks_pos):
    # Find the first block in the initial direction
    current_position = get_first_block_in_direction(initial_position, initial_direction, map_size, blocks_pos)
    current_direction = initial_direction
    route = []
    while current_position is not None:
        if (current_position, current_direction) in route:
            route.append((current_position, current_direction))
            route.append('LOOP')
            break  # Avoid infinite loops
        route.append((current_position, current_direction))
        current_position = blocks_dict[current_position][current_direction]
        current_direction = guard_direction_shift[current_direction]
    return route

In [98]:
def insert_new_block(map_matrix, new_block):
    new_map_matrix = map_matrix.copy()
    new_map_matrix[tuple(new_block)] = '#'
    new_blocks_pos = np.where(new_map_matrix == '#')
    return new_blocks_pos

In [97]:
full_route = get_full_route((40,40), '^', map_size, blocks_dict, blocks_pos)
full_route

[(np.int64(229), '^'),
 (np.int64(234), '>'),
 (np.int64(268), 'v'),
 (np.int64(261), '<')]

In [ ]:
# Prepare all candidate positions for the new block
block_candidate_positions = np.where(visited_map == 'X')
print(block_candidate_positions, len(block_candidate_positions[0]))

(array([  5,   5,   5, ..., 127, 128, 129]), array([64, 65, 66, ..., 73, 73, 73])) 5199


In [102]:
guard_position

(np.int64(89), np.int64(74))

In [106]:
total_loops = 0
for i in range(len(block_candidate_positions[0])):
    if i % 100 == 0:
        print(f'Processing candidate {i}/{len(block_candidate_positions[0])}')
    test_new_block = (block_candidate_positions[0][i], block_candidate_positions[1][i])
    new_blocks_pos = insert_new_block(map_matrix, test_new_block)
    new_blocks_dict = create_blocks_dict(new_blocks_pos)
    new_full_route = get_full_route(guard_position, '^', map_size, new_blocks_dict, new_blocks_pos)
    if 'LOOP' in new_full_route:
        total_loops += 1


print(total_loops)

Processing candidate 0/5199
Processing candidate 100/5199
Processing candidate 200/5199
Processing candidate 300/5199
Processing candidate 400/5199
Processing candidate 500/5199
Processing candidate 600/5199
Processing candidate 700/5199
Processing candidate 800/5199
Processing candidate 900/5199
Processing candidate 1000/5199
Processing candidate 1100/5199
Processing candidate 1200/5199
Processing candidate 1300/5199
Processing candidate 1400/5199
Processing candidate 1500/5199
Processing candidate 1600/5199
Processing candidate 1700/5199
Processing candidate 1800/5199
Processing candidate 1900/5199
Processing candidate 2000/5199
Processing candidate 2100/5199
Processing candidate 2200/5199
Processing candidate 2300/5199
Processing candidate 2400/5199
Processing candidate 2500/5199
Processing candidate 2600/5199
Processing candidate 2700/5199
Processing candidate 2800/5199
Processing candidate 2900/5199
Processing candidate 3000/5199
Processing candidate 3100/5199
Processing candidate